In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils

from torch.autograd import Variable
from torchvision import datasets, models, transforms

In [6]:
import numpy as np
import os

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(30),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [9]:
data_dir = 'data'

In [10]:
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'test']}

In [11]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32, shuffle=True) for x in ['train', 'test']}

In [15]:
dataset_sizes  = {x: len(image_datasets[x]) for x in ['train', 'test']}
dataset_sizes

{'train': 793, 'test': 207}

In [16]:
class_names = image_datasets['train'].classes
class_names

['cavalo',
 'comida',
 'dinossauro',
 'elefante',
 'flor',
 'indio',
 'montanha',
 'monumento',
 'onibus',
 'praia']

In [17]:
model = models.resnet34(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /Users/paulo/.torch/models/resnet34-333f7ec4.pth
100%|██████████| 87306240/87306240 [02:28<00:00, 705658.67it/s] 


In [18]:
criterion = nn.CrossEntropyLoss()

In [19]:
optimizer = optim.Adam(model.parameters(), lr=0.005)

In [20]:
def train(model, dataloader, optimizer, criterion, epoch):
    model.train()
    
    for batch_idx, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        prediction = model(x)

        loss = criterion(prediction, y)
        loss.backward()

        optimizer.step() 
        
        if batch_idx % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(x), len(dataloader.dataset),
                100. * batch_idx / len(dataloader), loss.data.item()))

In [21]:
def val(model, dataloader, criterion):
    model.eval()
    
    val_loss = 0
    correct = 0
    
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            
            prediction = model(x)
            val_loss += criterion(prediction, y).item()
            
            y_hat = prediction.max(1, keepdim=True)[1]
            correct += y_hat.eq(y.view_as(y_hat)).sum().item()
    
    val_loss /= len(dataloader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        val_loss, correct, len(dataloader.dataset),
        100. * correct / len(dataloader.dataset)))

In [22]:
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))

model.to(device);

In [23]:
for epoch in range(1, 50+1):
    train(model, dataloaders['train'], optimizer, criterion, epoch)
    val(model, dataloaders['val'], criterion)
    for param_group in optimizer.param_groups:
        param_group['lr'] *= 0.9

Train Epoch: 1 [0/793 (0%)]	Loss: 2.454503


KeyboardInterrupt: 